In [ ]:
! pip install ffmpy
! chmod +x /Users/sjegan/anaconda3/bin/ffmpeg
! pip install unrar
! pip install beautifulsoup4
! pip install urllib3


In [7]:

from os import listdir
import os
from os.path import isfile, join
import shutil

from unrar import rarfile
import os
import subprocess
import time
from os import listdir
import os
from os.path import isfile, join
import shutil
import glob, os

def convertImage(source, destination, small=False):
    import scipy
    imgpath = source

    import scipy.misc
    import numpy as np
    img = scipy.misc.imread(imgpath)
    assert len(img.shape) == 3
    w1 = img.shape[0]
    h1 = img.shape[1]
    h2 = 1024
    w2 = int((h1*h2)/w1)

    if small == True:
        if w2 > h2:
            h2 = 512
            w2 = int((h1*h2)/w1)
    img_resized = scipy.misc.imresize(img, (h2, w2))
    img_answer = (img_resized/255.0).astype('float32')

    import matplotlib.pyplot as plt
#     plt.imshow(np.uint8(img))
#     plt.show()
#     plt.imshow(np.uint8(img_resized))
#     plt.show()
    scipy.misc.imsave(destination, img_resized)

def convertAllImages(files, small=True):
    for f in files:
        convertImage(f, f, small)

def getListOfFiles(dirName, filterlist):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath, filterlist)
        else:
            canadd = False
            for f in filterlist:
                if fullPath.endswith(f):
                    allFiles.append(fullPath)
                    break
    allFiles.sort()
                
    return allFiles
def shrinkImages(dir_name):
    files = getListOfFiles(dir_name, ['jpg', 'jpeg', 'JPEG', 'JPG', 'png', 'PNG'])
    convertAllImages(files, False)

def tomp4(input_file, output_file):
    import ffmpy
    crf = 18
    input_name = input_file
    output_name = output_file
    inp={input_name:None}
    outp = {output_name:'-vcodec libx264 -crf 28 -preset faster -tune film -strict -2 -filter:v scale="640:trunc(ow/a/2)*2" -b:v 128k -b:a 100k -r 20 '}
    outp = {output_name:'-af apad -shortest -avoid_negative_ts make_zero -fflags +genpts -vcodec libx264 -acodec aac -preset faster -tune film -strict -2 -filter:v scale="640:trunc(ow/a/2)*2" -b:v 128k -b:a 100k -r 20 '}
    ff=ffmpy.FFmpeg(inputs=inp,outputs=outp)
    print(ff.cmd) # just to verify that it produces the correct ffmpeg command
    ff.run()

from os import listdir
import os
import ffmpy
from os.path import isfile, join
import shutil

def shrinkVideos(dir_name, dest_dir_name, bin_dir_name):
    onlyfiles = [(join(dir_name, f), f) for f in listdir(dir_name) if isfile(join(dir_name, f))]
    totalfiles = len(onlyfiles)
    onlyfiles.sort()
    i=1
    onlyfiles.sort
    for loc, filename in onlyfiles:
        if loc.endswith(".avi") or loc.endswith(".mp4") or loc.endswith(".MP4") or loc.endswith(".mkv") or loc.endswith(".mpg") or loc.endswith(".flv"):
            cmd_filelist = "C:\\Users\\Jegan\\Downloads\\ffmpeg\\bin\\ffmpeg.exe -i \"" + loc + "\" -filter:v scale=\"640:trunc(ow/a/2)*2\" -b:v 128k -b:a 100k -r 20 \"" + dest_dir_name + filename + ".mp4\""
            tomp4(loc, dest_dir_name + "/" + os.path.splitext(os.path.basename(filename))[0] + ".mp4")
            print(cmd_filelist)
            print(f'-------------------{i} / {totalfiles}----------------------')
            shutil.move(src=loc, dst=bin_dir_name)
            i = i + 1

from bs4 import BeautifulSoup
import urllib3
http = urllib3.PoolManager()
from bs4 import BeautifulSoup
import urllib3
import re

def downloadfile(url, path):
    import urllib3
    http = urllib3.PoolManager()
    r = http.request('GET', url, preload_content=False)

    with open(path, 'wb') as out:
        while True:
            data = r.read(1024 * 1024 * 30)
            if not data:
                break
            out.write(data)

    r.release_conn()


def downloadFromFTP(url, convert, extension):
    response = http.request('GET', url)
    print(response)
    soup = BeautifulSoup(response.data)
    links = []
    import os
    i = 0
    t = False

    for link in soup.findAll('a', attrs={'href': re.compile(extension)}):

        i = i + 1
        if '1-25.flv' == link.get('href'):
            t = True
        if i > 0 and link.get('href') not in links and t == True:
            links.append(link.get('href'))
            print(link.get('href'))
            downloadfile(url+link.get('href'), convert + os.path.basename( link.get('href')))


import os
import subprocess
import time
from os import listdir
import os
from os.path import isfile, join
import shutil



def concatenateFiles(dir_name, video_files_ffmpeg, extension, output_file, group=None):
    video_files = video_files_ffmpeg
    base_dir = dir_name

    files = []
    onlyfiles = [(join(dir_name, f), f) for f in listdir(dir_name) if isfile(join(dir_name, f))]
    onlyfiles.sort()
    for loc, filename in onlyfiles:
        if loc.endswith(extension):
            files.append(loc)
    files.sort()
    if group = None:
        group = len(files)
    i = 0
    tempfiles = []
    for f in files:
        tempfiles.append(f)
        i = i + 1

        if i % group == 0:
            file_list = open(video_files, "w")
            for tee in tempfiles:
                file_list.write("file '%s'\n" % tee)
            file_list.close()
            cmd = ["ffmpeg",
                   "-f",
                   "concat",
                   "-safe",
                   "0",
                   "-loglevel",
                   "quiet",
                   "-i",
                   "%s" % video_files,
                   "-c",
                   "copy",
                   "%s" % (output_file + str(i)+".mp4")
                   ]
            p = subprocess.Popen(cmd, stdin=subprocess.PIPE)
            fout = p.stdin
            fout.close()
            p.wait()
            print(p.returncode)
            tempfiles = []
    if len(tempfiles) > 0:
        file_list = open(video_files, "w")
        for tee in tempfiles:
            file_list.write("file '%s'\n" % tee)
        file_list.close()
        cmd = ["ffmpeg",
               "-f",
               "concat",
               "-safe",
               "0",
               "-loglevel",
               "quiet",
               "-i",
               "%s" % video_files,
               "-c",
               "copy",
               "%s" % (output_file + str(i)+".mp4")
               ]
        p = subprocess.Popen(cmd, stdin=subprocess.PIPE)
        fout = p.stdin
        fout.close()
        p.wait()
        print(p.returncode)
        tempfiles = []
        


In [ ]:

shrinkImages("/Users/sjegan/Downloads/uuu/Batman Damned")


In [6]:

dir_name = "/Users/sjegan/Downloads/untitled folder 3"
dest_dir_name = "/Users/sjegan/Downloads/dest"
bin_dir_name = "/Users/sjegan/Downloads/dest/del"
shrinkVideos(dir_name, dest_dir_name, bin_dir_name)


SyntaxError: invalid syntax (<ipython-input-6-3914fedef23a>, line 2)

In [ ]:

url = 'http://193.169.52.206/a-aladdin/'
convert = "/Users/sjegan/Downloads/com2/"
extension = "flv"
downloadFromFTP(url, convert, extension)


In [ ]:

dir_name = "/Users/sjegan/Downloads/concat"

video_files = "/Users/sjegan/Downloads/video_list.txt"
output_file = "/Users/sjegan/Downloads/shaun"
extension = ".mp4"

concatenateFiles(dir_name, video_files_ffmpeg, extension, output_file, group=None)
